# electricity_market_env

> Fill in a module description here

In [19]:
#| default_exp electricity_market_env

In [20]:
#| hide
from nbdev.showdoc import *

In [21]:
#| export
import gymnasium as gym
import numpy as np
from gymnasium.envs.registration import register

In [22]:
#| export
class ElectricityMarketEnv(gym.Env):
    def __init__(self, env_config=None):
        # Environment Configuration
        if env_config is None:
            env_config = {}
        self._battery_capacity = env_config.get('battery_capacity', 100)
        self._config = env_config

        self.action_space = gym.spaces.Box(low=-self._battery_capacity, high=self._battery_capacity, shape=(1,), dtype=np.float64)
        self.observation_space = gym.spaces.Box(low=np.array([0, 0, 0]),
                                                high=np.array([self._battery_capacity, np.inf, np.inf]),
                                                shape=(3,), dtype=np.float64)


        # State of Environment
        self._current_state_of_charge = env_config.get('init_state_of_charge', 100)
        self._timestep = 0

    def _is_action_valid(self, action) -> bool:
        target_state_of_charge = self._current_state_of_charge + action
        return 0 < target_state_of_charge < self._battery_capacity

    def step(self, action):
        action = action[0]
        self._timestep += 1
        done = False
        truncated = False

        if not self._is_action_valid(action):
            return self._get_obs(), 0, done, truncated, {}

        reward = self._calculate_reward(action)
        self._current_state_of_charge += action


        observation = self._get_obs()

        return observation, reward, done, truncated, {}

    def _calculate_reward(self, action) -> float:
        return  (action - self._demand_of_electricity()) * self._price()

    def _demand_of_electricity(self) -> float:
        noise_std = 5
        demand = 120 * np.exp(-((self._timestep - 0.4) ** 2) / (2 * (0.05 ** 2))) + 100 * np.exp(-((self._timestep - 0.7) ** 2) / (2 * (0.1 ** 2)))
        noise = np.random.normal(0, noise_std)
        return demand + noise


    def _price(self) -> float:
        noise_std = 5
        price = 120 * np.exp(-((self._timestep - 0.4) ** 2) / (2 * (0.05 ** 2))) + 100 * np.exp(-((self._timestep - 0.7) ** 2) / (2 * (0.1 ** 2)))
        noise = np.random.normal(0, noise_std)
        return price + noise

    def reset(self, *, seed=None, options=None, **kwargs):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        self._current_state_of_charge = self._config.get('init_state_of_charge', 100)
        self._timestep = 0

        observation = self._get_obs()
        return observation, {}

    def _get_obs(self):
        return np.array([self._current_state_of_charge, float(self._demand_of_electricity()), float(self._price())])



In [23]:
#| export
register(
    id='ElectricityMarketEnv-v0',
    entry_point='electricity_market.electricity_market_env:ElectricityMarketEnv',
)

In [24]:
#| hide
import nbdev; nbdev.nbdev_export()